In [41]:
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew

In [42]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"]='gpt-4-turbo'

In [43]:
sales_rep_agent = Agent(
    role = "Sales Representative",
    goal = "Identify high quality leads that meet "
    "our ideal customer profile",
    backstory=(
        "As part of a dynamic sales team at CrewAI, " 
        "your mission is to scour the digital landscape " 
        "for potential leads. Armed with cutting edge "
        "tools and a strategic mindset, you analyze data, "
        "trends, and interactions to unearth opportunities"
        "that others might overlook. Your work is crucial "
        "in paving the way for meaningful engagements "
        "and driving the company's growth."

    ),
    allow_delegation=False,
    verbose=True
)

In [44]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [45]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool, BaseTool

In [46]:
directory_read_tool = DirectoryReadTool(directory="https://s172-29-74-207p8888.lab-aws-production.deeplearning.ai/tree/L4/instructions")
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [47]:
class SentimentAnalysisTool(BaseTool):
    name: str= "Sentiment Analysis Tool"
    description: str= ("Analyzes the sentiment of text"
                       "to ensure positive and engaging communication.")
    def _run(self, text: str) -> str:
        #custom code
        return "positive"

In [48]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [49]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent
)

In [54]:
personalized_outreach_task= Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
tools=[sentiment_analysis_tool, search_tool],
agent=lead_sales_rep_agent,)

In [55]:
crew = Crew(
    agents = [sales_rep_agent, lead_sales_rep_agent],
    tasks = [lead_profiling_task, personalized_outreach_task],
    verbose=True,
    memory=True
)

2024-10-21 15:09:59,844 - 23124 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [57]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result=crew.kickoff(inputs=inputs)

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


# Agent: Sales Representative
## Thought: I need to gather comprehensive and detailed information on DeepLearningAI. Searching the internet is the best action to collect initial information about the company, their key decision-makers, recent milestones, and possible needs relevant to our products.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"DeepLearningAI company profile\"}"
## Tool Output: 

Search results: Title: Deeplearning.ai - Crunchbase Company P

In [58]:
from IPython.display import Markdown
Markdown(result.raw)

Here are the tailored email drafts for Andrew Ng, CEO of DeepLearningAI:

**Email 1: Enhancing Educational Content with AI-Powered Personalization**
Subject: Transform DeepLearningAI's Learner Experience with Advanced AI Personalization

Dear Andrew,

I hope this message finds you well. I'm reaching out to commend you on your remarkable success in impacting over 7 million learners through DeepLearningAI. Your commitment to expanding the frontiers of AI education is truly inspiring.

At CrewAI, we specialize in tailoring AI solutions that can significantly elevate the learning experience. Given your goal to enhance content and platform capabilities, I believe our AI-Powered Recommendation Engines would seamlessly integrate with your services, offering personalized course suggestions that adapt to individual learning patterns and objectives. This targeted approach does not only enrich the learning journey but also boosts engagement and retention, furthering your mission of accessible AI education.

Would you be available for a brief meeting to explore how we can collaborate toward enhancing DeepLearningAI's offerings? I'm looking forward to discussing how we can bring this vision to life and support your ongoing success.

Best regards,
[Your Name]

**Email 2: Supporting Growth with Scalable Cloud Solutions**
Subject: Scaling DeepLearningAI Seamlessly with CrewAI Cloud Solutions

Hello Andrew,

It's been exciting to see DeepLearningAI achieve such incredible growth, reaching millions worldwide and setting high standards in AI learning platforms. As you continue to expand, the need for a robust infrastructure that can support your growing learner base becomes increasingly critical.

At CrewAI, we provide Scalable Cloud Solutions that can help you efficiently manage the increased load of online learners and data. These solutions are designed to ensure smooth, uninterrupted access to your educational content, thereby enhancing user satisfaction and platform reliability.

Could we schedule a call to discuss how our cloud solutions can align with your expansion plans and contribute to an even more successful learner experience?

Looking forward to your thoughts.

Warm regards,
[Your Name]

**Email 3: Driving User Engagement with Advanced Analytics**
Subject: Unlock Insights and Drive Engagement at DeepLearningAI with CrewAI Analytics

Dear Andrew,

Firstly, congratulations on your tremendous success with DeepLearningAI. As you strive to create a more engaging and effective learning environment, understanding learner behavior and course interaction becomes essential.

Our Advanced Data Analytics tools are designed to provide deep insights into learning patterns, enabling you to refine courses and tailor marketing strategies effectively. These analytics can be a game-changer in how you adapt and evolve your offerings to meet learner needs more precisely.

I would love to discuss how our analytics tools can integrate with your platform to help you achieve better outcomes and further your vision of democratizing AI education.

Please let me know a convenient time for a conversation.

Best regards,
[Your Name]

**Signatures have been generalized with [Your Name], and should be replaced with the actual name of the sender before dispatching these emails.